Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menubar, select Cell$\rightarrow$Run All).

Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE", as well as your name and collaborators below:

In [ ]:
NAME = "Ilya Grebenkin"
COLLABORATORS = "-"

---

# I. $LU$ factorization of a square matrix

Consider a simple naive implementation of the LU decomposition. 

Note that we're using the `numpy` arrays to represent matrices [do **not** use `np.matrix`].

In [ ]:
import numpy as np

def diy_lu(a):
    """Construct the LU decomposition of the input matrix.
    
    Naive LU decomposition: work column by column, accumulate elementary triangular matrices.
    No pivoting.
    """
    N = a.shape[0]
    
    u = a.copy()
    L = np.eye(N)
    for j in range(N-1):
        lam = np.eye(N)
        gamma = u[j+1:, j] / u[j, j]
        lam[j+1:, j] = -gamma
        u = lam @ u

        lam[j+1:, j] = gamma
        L = L @ lam
    return L, u

In [ ]:
# Now, generate a full rank matrix and test the naive implementation
import numpy as np

N = 6
a = np.zeros((N, N), dtype=float)
for i in range(N):
    for j in range(N):
        a[i, j] = 3. / (0.6*i*j + 1)

L,U = diy_lu(a)

np.linalg.matrix_rank(a)

In [ ]:
# Tweak the printing of floating-point numbers, for clarity
np.set_printoptions(precision=3)

In [ ]:
L, u = diy_lu(a)
print(L, "\n")
print(u, "\n")

# Quick sanity check: L times U must equal the original matrix, up to floating-point errors.
print(a-L@u)

# II. The need for pivoting

Let's tweak the matrix a little bit, we only change a single element:

In [ ]:
a1 = a.copy()
a1[1, 1] = 3
print(a1)

Resulting matix still has full rank, but the naive LU routine breaks down.

In [ ]:
np.linalg.matrix_rank(a1)

In [ ]:
l, u = diy_lu(a1)

print(l, u, sep='\n')

### Test II.1

For a naive LU decomposition to work, all leading minors of a matrix should be non-zero. Check if this requirement is satisfied for the two matrices `a` and `a1`.

Note that directly checking if a determinant equals zero is not recommended (both computing a determinant explicitly, *and* comparing a floating-point number to zero are not recommended). 

Instead, check if leading minors have full rank (use `np.linalg.matrix_rank` function).

In [ ]:
def minor(a):
    ''' Check if all leading minors have full rank.
    
    Parameters
    ----------
    a : np.array
        2D array representing the square matrix
    
    Returns
    -------
    bool
        True if all leading minors have full rank, False otherwise
    '''
    # YOUR CODE HERE
    raise NotImplementedError()

In [ ]:
assert minor(a) is True
assert minor(a1) is False


### Test II.2

Modify the `diy_lu` routine to implement column pivoting. Keep track of pivots, you can either construct a permutation matrix, or a swap array (your choice).

Implement a function to reconstruct the original matrix from a decompositon. Test your routines on the matrices `a` and `a1`.

In [ ]:
def diy_lu_mod(a):
    '''Perform pivoted LU factorization of the input matrix. 
    
    Parameters
    ----------
    a : np.array
        2D array representing a square matrix with float entries
       
    Returns
    -------
    P, L, U : ndarrays
        factors. Here P is a permutation matrix, L is lower triangular
        with unit diagonal elements, and U upper triangular.
    '''
    # YOUR CODE HERE
    raise NotImplementedError()

In [ ]:
from numpy.testing import assert_allclose

P, L, U = diy_lu_mod(a)
assert_allclose(P.T @ P, 
                np.identity(a.shape[0]), atol=1e-14)
assert_allclose(a,
                P @ L @ U, atol=1e-10)

P1, L1, U1 = diy_lu_mod(a1)
assert_allclose(P1.T @ P1,
                np.identity(a1.shape[0]), atol=1e-14)
assert_allclose(a1,
                P1 @ L1 @ U1, atol=1e-10)


# III. Use the LU factorization to solve a linear system

Write a function for solving a system of linear equations $\mathrm{A} \vec{x} = \vec{b}$ using the pivoted LU factorization.

Use your `diy_lu_mod` routine to factorize the input matrix. You may also use `scipy.linalg.solve_triagular` for solving systems of equations with triangular matrices $L$ and $U$. Do *not* use general-purpose library `solve` routines.

(In case you wonder why bothering: what is the computational complexity of solving a linear system with a triangular matrix? With a general matrix?)

In [ ]:
from scipy.linalg import solve_triangular

def lu_solve(a, b):
    """Solve a linear system `a @ x = b` via pivoted LU decomposition.

   Parameters
    ----------
    a : np.array
        2D array representing a square matrix with float entries
    b : np.array
        1D array representing the right-hand-side of the system
       
    Returns
    -------
    x : np.array
        The solution of `a @ x = b`
    """
    P, L, U = diy_lu_mod(a)
    # YOUR CODE HERE
    raise NotImplementedError()

In [ ]:
b = np.arange(a.shape[0])
assert_allclose(lu_solve(a, b),
                np.linalg.solve(a, b), atol=1e-10)


### Extra points : multiple r.h.s

Check that your `lu_solve` function handles the case where the `b` array contains an $M$-times-$N$ matrix, where each column represents a right-hand-side of a linear system. Modify the `lu_solve` function if necessary. 

For instance, you can compute the inverse matrix of `a2` in the cell below.

In [ ]:
assert_allclose(lu_solve(a2, np.identity(a2.shape[0])),
                np.linalg.inv(a2), atol=1e-10
               )